# Purpose : Read PDF dir , do rag, use LangChain LCEL , use OpenAI to generate Question Answer 

Steps
1. Read pdf dir
2. Ebedding using HuggingFace
3. Chroma DB as Vertor DataBase
4. Do RAG 
5. Save the output in html format
6. Use OpenAI to generate Answer

## PDF Read 
## Store in Chroma DB
## Use Retriever to do Question / Answer

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
import chromadb
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents.base import Document
import os
import json
import pandas as pd


#os.environ['OPENAI_API_KEY']='Put your API key here'

In [ ]:
def GetChromaDB(pdfdirpath:str,collectionname:str)->Chroma:
    pdf_folder_path = pdfdirpath
    documents = []
   
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)  
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    chunked_documents = text_splitter.split_documents(documents)
    print('load done')
    client = chromadb.Client()
    collection = client.get_or_create_collection("python")
    
    #embeddings = HuggingFaceEmbeddings()
    embeddings = OpenAIEmbeddings()
    
    vectordb = Chroma.from_documents(
        documents=chunked_documents,
        embedding=embeddings,
        persist_directory="D:\\ChromaDB\\chroma_store\\",
        collection_name=collectionname
    )
    
    return vectordb

In [30]:
vectorDB = GetChromaDB(pdfdirpath="D:\\doc",collectionname="python")

load done


In [31]:
retriever =vectorDB.as_retriever()

In [34]:
#query='What is a function in phthon?'
query='What is a lambda?'

result = retriever.invoke(query)
print(result)

[Document(page_content='K\nkey—An item that is associated with a value \nand which is used to locate that value in a \ncollection.\nkeypunch machine —An early input device \nthat allowed the user to enter programs and data onto punched cards.\nkeys —Resources used to encrypt or decrypt \ndata.\nkeyword parameter —A type of parameter \nthat allows the programmer to specify the name of the parameter and its value at the point of a function or method call.\nl\nlabel object —A window object that displays \ntext or an image, usually to describe the roles of other window objects.\nlambda —The mechanism by which an anony-\nmous function is created.\nlifetime —The time during which a data object \nor method call exists.\nlinear —A function expressing a rate of growth \nof work in direct proportion to the size of a problem.\nlinear loop structure —A loop that does not \ncontain a nested loop.\nlinear search —A type of search that examines \neach value in a sequence, until a target value is foun

In [35]:
docs_dict =[{"page_content":doc.page_content,"metadata":doc.metadata} for doc in result]
df = pd.json_normalize(data=docs_dict)
df.to_html('rag.html')

## Now we will use Chain & LLM (OpenAI) to give us the output of RAG

In [36]:
from langchain_chroma import Chroma
import chromadb
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [37]:
_client = chromadb.PersistentClient(path="D:\\ChromaDB\\chroma_store\\")
print(_client.list_collections())

[Collection(name=python), Collection(name=langchain)]


In [38]:
_embeddings = OpenAIEmbeddings()

In [39]:
_chromaDB = Chroma(collection_name="python",embedding_function=_embeddings,persist_directory="D:\\ChromaDB\\chroma_store\\")
_retriever =Chroma.as_retriever(_chromaDB)

In [48]:
#query='What is a function in phthon?'
query='What is a lambda function in python?'


result = _retriever.invoke(query)
print(result)

[Document(page_content="Chapter 3\nFunctions\nIn the context of programming, a function is a named sequence of statements that performs\na computation. When you deﬁne a function, you specify the name and the sequence of\nstatements. Later, you can “call” the function by name.\n3.1 Function calls\nWe have already seen one example of a function call :\n>>> type(42)\n<class 'int'>\nThe name of the function is type . The expression in parentheses is called the argument of\nthe function. The result, for this function, is the type of the argument.\nIt is common to say that a function “takes” an argument and “returns” a result. The result\nis also called the return value .\nPython provides functions that convert values from one type to another. The intfunction\ntakes any value and converts it to an integer, if it can, or complains otherwise:\n>>> int( '32')\n32\n>>> int( 'Hello ')\nValueError: invalid literal for int(): Hello\nintcan convert ﬂoating-point values to integers, but it doesn’t ro

In [41]:
_llm = ChatOpenAI()

In [51]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer as concise as possible. 

{context}
Question: {question}
Helpful Answer:"""


rag_prompt_custom = PromptTemplate.from_template(template)
#prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
     {"context": retriever | format_docs , "question": RunnablePassthrough()} 
     | rag_prompt_custom 
     | _llm
     | StrOutputParser()
     )

In [52]:
_result = rag_chain.invoke(query)
type(_result)

str

In [53]:
_result

'A lambda function in Python is a small anonymous function defined with the lambda keyword. It can have any number of arguments but only one expression. Lambda functions can be used wherever function objects are required.'